In [ ]:
using Pkg; Pkg.activate(".")
using CairoMakie, LinearAlgebra, LaTeXStrings, ForwardDiff, FFTA
include("spectral_tools.jl")

# Fourier Spectral Methods (1D)

## Revise Motivating Example 

We return to the boundary value problem with periodic boundary conditions: 
$$
\begin{aligned}
  - u'' + u &= f, \quad x \in (-\pi, \pi), \\ 
  u(-\pi) &= u(\pi), \\ 
  u'(-\pi) &= u'(\pi)
\end{aligned}
$$


In [ ]:
# (1) let's take our periodic agnesi example as f
f_fun, α = let c = 3
    (x -> 1 / (1 + c^2 * sin(x/2)^2)), 2*asinh(1/c)
end

# (2) before we constructed the projection, now we use the 
#     trigonometric interpolant 
N = 10
F̂ = triginterp_fft(f_fun, N)

# (3) solve the PDE in reciprocal space
#     note Û represents the solution. We are done here!
K = kgrid(N)
Û = F̂ ./ (1 .+ abs.(K).^2)
;

In [ ]:
# (4) To plot the solution we just evaluate it on a 
# refined grid 

Up, Xp = evaltrig_grid(Û, N)
up, xp = evaltrig_grid(Û, 128)

fig1 = Figure(size=(400, 250)); ax1 = Axis(fig1[1,1])
lines!(ax1, xp, up, label = L"u_N(x)")
scatter!(ax1, Xp, Up, label = "nodal vals")
axislegend(ax1, position = :ct)
fig1

In [ ]:
# define a known exact solution and compute the resulting f
u_ex, α = let c = 3
    (x -> 1 / (1 + c^2 * sin(x)^2)), asinh(1/c)
end
du_ex = x -> ForwardDiff.derivative(u_ex, x)
f_fun = x -> u_ex(x) - ForwardDiff.derivative(du_ex, x)

function solve_and_err(N; Ne = 1024)
    K = kgrid(N)
    F̂ = triginterp_fft(f_fun, N)
    Û = F̂ ./ (1 .+ K.^2)
    Xe = xgrid(Ne); Ue = u_ex.(Xe); ∇Ue = du_ex.(Xe)
    U, _ = evaltrig_grid(Û, Ne)
    ∇U, _ = evaltrig_grid(im * K .* Û, Ne)
    return norm(U - Ue, Inf), norm(∇U - ∇Ue, Inf)
end

# looking at errors for the spectral method
NN = 8:8:128
errs = solve_and_err.(NN)
errs0 = [e[1] for e in errs]
errs1 = [e[2] for e in errs]
;

In [ ]:
fig2 = Figure(size=(400, 300))
ax2 = Axis(fig2[1,1], xscale = identity, yscale = log10, xlabel = L"N", ylabel = "error")
scatterlines!(ax2, NN, errs0, label = L"\Vert u_N - u\Vert_\infty")
scatterlines!(ax2, NN, errs1, label = L"\Vert u_N' - u'\Vert_\infty")
lines!(ax2, NN[6:end-4], 10*exp.(- α * NN[6:end-4]), linestyle=:dash, color = :black, label = L"\sim e^{-\alpha N}")
axislegend(ax2, position=:rt)
fig2

---

### Preview: A Transport Problem

$$
	u_t + c u_x = 0 \qquad \text{+ PBC}
$$
For now we take $c$ to be constant. Below we will switch to $x$-dependent $c$.

First discretise in time using the Leapfrog scheme 
$$
	\frac{u^{n+1} - u^{n-1}}{2 \Delta t} + c (u^n)_x = 0.
$$

We will also compare this to a finite-difference scheme.

In [ ]:
# ------------------------------------------
# Problem setup
N = 64  
dt = π/(6N) 
tmax = 32.0
c = 1.0
u0 = x ->  exp(- 10 * (1+cos(x))/2 )
#------------------------------------------

X = xgrid(N)
K = kgrid(N)

# differentiation operator in Fourier space 
D̂ = im*K

# initial condition, we also need one additional v in the past
# (this takes one step of the PDE backward in time)
V = u0.(X)
Vold = V + dt * c * real.( ifft( D̂ .* fft(V) ) ) 

# Setup for the FD Scheme, UPWIND D1 and CENTERED D2
h = π/N
D1 = collect(Bidiagonal(ones(2*N)/h, -ones(2*N-1)/h, :L))
D1[1, 2*N] = -1/h # (PBC)
D2 = collect(Tridiagonal(-ones(2*N-1)/(2h), zeros(2*N), ones(2*N-1)/(2h)))
D2[1,2*N] = -1/(2h); D2[2*N,1] = 1/(2h)
V1h = V; V2h = V; V2hold = Vold

# ------------- setup the plotting for an animation

t_obs = Observable(0.0)
V_obs = Observable(V)
V1h_obs = Observable(V1h)
V2h_obs = Observable(V2h)

fig = Figure(size = (500, 250))
ax = Axis(fig[1,1], title = "t = $(round(t_obs, digits=2))")
limits!(ax, 0.0, 2*π, -0.1, 1.2)
lines!(ax, X, (@lift u0.(X .- $t_obs)), label="exact")
lines!(ax, X, V_obs, label="spectral")
lines!(ax, X, V1h_obs, label="upwind", linestyle=:dash)
lines!(ax, X, V2h_obs, label="centered", linestyle=:dash)
Legend(fig[1,2], ax)

record(fig, "animation.gif", 0:dt:tmax; framerate=60) do t
    global V, Vold, V1h, V2h, V2hold, D1, D2, D̂, obs
    # SPECTRAL 
    # differentiation in reciprocal space
    W = real.( ifft( D̂ .* fft(V) ) )   
    # multiplication and update in real space
    V, Vold = Vold - 2 * dt * c * W, V

    # FINITE-DIFFERENCE
    V1h = V1h - dt * c * (D1 * V1h)  # (upwind)
    V2h, V2hold = V2hold - 2*dt * c * (D2*V2h), V2h     #(centered)  
    
    # update the plot 
    t_obs[] = t 
    V_obs[] = V 
    V1h_obs[] = V1h
    V2h_obs[] = V2h
end

In [ ]:
display("text/html", "<img src='animation.gif' width=400px>")

In [ ]:
# plot the final solution again to see the numerical damping and dispersion

fig = Figure(size = (500, 250))
ax = Axis(fig[1,1], title = "t = $tmax")
limits!(ax, 0.0, 2*π, -0.1, 1.2)
lines!(ax, X, u0.(X .- tmax), label="exact")
lines!(ax, X, V, label="spectral")
lines!(ax, X, V1h, label="upwind", linestyle=:dash)
lines!(ax, X, V2h, label="centered", linestyle=:dash)
Legend(fig[1,2], ax)
fig